# Amazon Spatial Model with Energy-Constrained Tipping Cascades

**Author:** Jason Holt  
**Date:** December 2025

## Overview

This notebook applies the energy-constrained tipping cascade framework to **spatially-explicit Amazon moisture recycling data** provided by Dr. Arie Staal (Utrecht University).

### Data Source

- **Dataset**: Amazon Adaptation Model (1° resolution)
- **Reference**: Wunderling et al. (2022) PNAS, https://doi.org/10.1073/pnas.2120777119
- **Provider**: Dr. Arie Staal, Utrecht University
- **Figshare**: https://figshare.com/articles/software/Amazon_Adaptation_Model/20089331

### Key Questions

1. How does moisture recycling create **spatial coupling** between Amazon grid cells?
2. Can we track **energy flow** through the moisture recycling network?
3. Does the **Lévy tunneling effect** apply to spatially-explicit cascades?
4. Which regions are most **thermodynamically vulnerable** to cascade propagation?

## 1. Setup and Data Loading

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob

# Path setup for k3s vs local
k3s_path = Path('/opt/research-local/src')
local_path = Path.cwd().parent / 'src'

if k3s_path.exists():
    sys.path.insert(0, str(k3s_path))
    project_root = Path('/opt/research-local')
    data_root = Path('/opt/research-local/data/amazon/amazon_adaptation_model')
    print("Running in k3s JupyterLab pod")
else:
    sys.path.insert(0, str(local_path))
    project_root = Path.cwd().parent
    data_root = project_root / 'data' / 'amazon' / 'amazon_adaptation_model'
    print("Running locally")

# Check data availability
if data_root.exists():
    print(f"Data found at: {data_root}")
else:
    print(f"WARNING: Data not found at {data_root}")
    print("Please download from: https://figshare.com/articles/software/Amazon_Adaptation_Model/20089331")

# Import energy-constrained module
from energy_constrained import (
    EnergyConstrainedCusp,
    EnergyConstrainedNetwork,
    GradientDrivenCoupling,
    energy_constrained_euler_maruyama,
    EnergyAnalyzer,
)

# Plot settings
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 11

print("\nEnergy-constrained module loaded successfully!")

In [ ]:
# Load netCDF data
try:
    from netCDF4 import Dataset
    NETCDF_AVAILABLE = True
except ImportError:
    print("netCDF4 not available, trying xarray...")
    try:
        import xarray as xr
        NETCDF_AVAILABLE = False
    except ImportError:
        raise ImportError("Please install netCDF4 or xarray: pip install netCDF4 xarray")

# Load sample data file
sample_file = data_root / 'average_network' / 'era5_new_network_data' / '1deg_2010_01.nc'

if NETCDF_AVAILABLE:
    ds = Dataset(sample_file)
    lat = ds.variables['lat'][:]
    lon = ds.variables['lon'][:]
    rain = ds.variables['rain'][:]
    evap = ds.variables['evap'][:]
    network = ds.variables['network'][:, :]
    ds.close()
else:
    ds = xr.open_dataset(sample_file)
    lat = ds['lat'].values
    lon = ds['lon'].values
    rain = ds['rain'].values
    evap = ds['evap'].values
    network = ds['network'].values
    ds.close()

print(f"Loaded data for January 2010")
print(f"  Grid cells: {len(lat)}")
print(f"  Network shape: {network.shape}")
print(f"  Lat range: [{lat.min():.1f}, {lat.max():.1f}]")
print(f"  Lon range: [{lon.min():.1f}, {lon.max():.1f}]")
print(f"  Rain range: [{rain.min():.1f}, {rain.max():.1f}] mm")
print(f"  Evap range: [{evap.min():.1f}, {evap.max():.1f}] mm")

## 2. Visualize the Moisture Recycling Network

In [ ]:
# Analyze network structure
print("Moisture Recycling Network Analysis")
print("=" * 50)

# Non-zero connections
n_connections = np.count_nonzero(network)
n_possible = network.shape[0] * network.shape[1] - network.shape[0]  # Exclude diagonal
density = n_connections / n_possible

print(f"Total grid cells: {network.shape[0]}")
print(f"Non-zero connections: {n_connections}")
print(f"Network density: {density:.4f}")

# Connection strength distribution
nonzero_values = network[network > 0]
print(f"\nMoisture flow (mm/month) statistics:")
print(f"  Min: {nonzero_values.min():.2f}")
print(f"  Max: {nonzero_values.max():.2f}")
print(f"  Mean: {nonzero_values.mean():.2f}")
print(f"  Median: {np.median(nonzero_values):.2f}")

# In-degree and out-degree
in_degree = np.sum(network > 0, axis=0)  # Columns: moisture received
out_degree = np.sum(network > 0, axis=1)  # Rows: moisture sent

print(f"\nDegree statistics:")
print(f"  Mean in-degree: {in_degree.mean():.1f}")
print(f"  Mean out-degree: {out_degree.mean():.1f}")
print(f"  Max in-degree: {in_degree.max()}")
print(f"  Max out-degree: {out_degree.max()}")

In [ ]:
# Spatial visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. Rainfall map
ax1 = axes[0, 0]
sc1 = ax1.scatter(lon, lat, c=rain, cmap='Blues', s=5, alpha=0.8)
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title('Rainfall (mm/month)')
plt.colorbar(sc1, ax=ax1)

# 2. Evapotranspiration map
ax2 = axes[0, 1]
sc2 = ax2.scatter(lon, lat, c=evap, cmap='Greens', s=5, alpha=0.8)
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')
ax2.set_title('Evapotranspiration (mm/month)')
plt.colorbar(sc2, ax=ax2)

# 3. In-degree (moisture received)
ax3 = axes[1, 0]
sc3 = ax3.scatter(lon, lat, c=in_degree, cmap='YlOrRd', s=5, alpha=0.8)
ax3.set_xlabel('Longitude')
ax3.set_ylabel('Latitude')
ax3.set_title('In-Degree (# sources)')
plt.colorbar(sc3, ax=ax3)

# 4. Out-degree (moisture sent)
ax4 = axes[1, 1]
sc4 = ax4.scatter(lon, lat, c=out_degree, cmap='YlOrRd', s=5, alpha=0.8)
ax4.set_xlabel('Longitude')
ax4.set_ylabel('Latitude')
ax4.set_title('Out-Degree (# destinations)')
plt.colorbar(sc4, ax=ax4)

plt.tight_layout()
plt.show()

print("\nKey observation: Western Amazon has high in-degree (receives moisture)")
print("Eastern/coastal areas have high out-degree (moisture sources)")

In [ ]:
# Network structure histograms
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Moisture flow distribution (log scale)
ax1 = axes[0]
ax1.hist(nonzero_values, bins=50, color='blue', alpha=0.7, edgecolor='black')
ax1.set_xlabel('Moisture Flow (mm/month)')
ax1.set_ylabel('Count')
ax1.set_title('Moisture Flow Distribution')
ax1.set_yscale('log')
ax1.grid(True, alpha=0.3)

# In-degree distribution
ax2 = axes[1]
ax2.hist(in_degree, bins=30, color='orange', alpha=0.7, edgecolor='black')
ax2.set_xlabel('In-Degree')
ax2.set_ylabel('Count')
ax2.set_title('In-Degree Distribution')
ax2.grid(True, alpha=0.3)

# Out-degree distribution
ax3 = axes[2]
ax3.hist(out_degree, bins=30, color='green', alpha=0.7, edgecolor='black')
ax3.set_xlabel('Out-Degree')
ax3.set_ylabel('Count')
ax3.set_title('Out-Degree Distribution')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Create Energy-Constrained Subnetwork

The full 567-cell network is computationally expensive. We'll create a representative subnetwork of the most connected cells for energy tracking experiments.

In [ ]:
def create_amazon_subnetwork(network, lat, lon, rain, evap, n_cells=50, min_flow=5.0):
    """
    Create an energy-constrained subnetwork from the Amazon moisture recycling data.
    
    Parameters
    ----------
    network : ndarray
        Full moisture recycling matrix (567x567)
    lat, lon : ndarray
        Coordinates
    rain, evap : ndarray
        Rainfall and evapotranspiration
    n_cells : int
        Number of cells to include in subnetwork
    min_flow : float
        Minimum moisture flow to include coupling (mm/month)
    
    Returns
    -------
    EnergyConstrainedNetwork
        Subnetwork with energy tracking
    """
    # Select cells with highest total moisture flow (in + out)
    total_flow = np.sum(network, axis=0) + np.sum(network, axis=1)
    top_indices = np.argsort(total_flow)[-n_cells:]
    
    # Create subnetwork
    sub_network = network[np.ix_(top_indices, top_indices)]
    sub_lat = lat[top_indices]
    sub_lon = lon[top_indices]
    sub_rain = rain[top_indices]
    sub_evap = evap[top_indices]
    
    # Create energy-constrained network
    net = EnergyConstrainedNetwork()
    
    # Add elements
    for i in range(n_cells):
        # Use rain/evap ratio as proxy for forest state
        # Higher ratio = more stable forest
        ratio = sub_rain[i] / (sub_evap[i] + 1e-6)
        
        # Scale barrier height by stability
        barrier = 0.3 + 0.4 * min(ratio / 2.0, 1.0)  # Range [0.3, 0.7]
        
        # Dissipation rate inversely related to forest health
        dissipation = 0.05 + 0.1 * (1 - min(ratio / 2.0, 1.0))
        
        element = EnergyConstrainedCusp(
            a=-1, b=1, c=0,
            E_stable=0.0,
            E_tipped=1.0,
            barrier_height=barrier,
            dissipation_rate=dissipation,
            heat_capacity=1.0
        )
        
        # Store metadata
        element.lat = sub_lat[i]
        element.lon = sub_lon[i]
        element.rain = sub_rain[i]
        element.evap = sub_evap[i]
        element.original_idx = top_indices[i]
        
        net.add_element(f'cell_{i}', element)
    
    # Add couplings based on moisture flow
    n_couplings = 0
    for i in range(n_cells):
        for j in range(n_cells):
            if i != j and sub_network[i, j] > min_flow:
                # Coupling strength proportional to moisture flow
                # Normalize by typical flow magnitude
                strength = sub_network[i, j] / 100.0  # Scale factor
                
                coupling = GradientDrivenCoupling(conductivity=strength)
                net.add_coupling(f'cell_{j}', f'cell_{i}', coupling)  # j sends to i
                n_couplings += 1
    
    print(f"Created subnetwork:")
    print(f"  Cells: {n_cells}")
    print(f"  Couplings: {n_couplings}")
    print(f"  Density: {n_couplings / (n_cells * (n_cells - 1)):.3f}")
    
    return net, top_indices, sub_lat, sub_lon

# Create subnetwork
amazon_net, selected_indices, sub_lat, sub_lon = create_amazon_subnetwork(
    network, lat, lon, rain, evap, n_cells=50, min_flow=5.0
)

In [ ]:
# Visualize selected cells
fig, ax = plt.subplots(figsize=(12, 8))

# All cells (gray)
ax.scatter(lon, lat, c='lightgray', s=3, alpha=0.5, label='All cells')

# Selected cells (colored by barrier height)
barriers = [amazon_net.get_element(f'cell_{i}').barrier_height for i in range(50)]
sc = ax.scatter(sub_lon, sub_lat, c=barriers, cmap='RdYlGn', s=50, 
                edgecolor='black', linewidth=0.5, label='Subnetwork')

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Amazon Subnetwork (colored by barrier height = stability)')
plt.colorbar(sc, label='Barrier Height')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Run Energy-Tracked Cascade Simulation

In [ ]:
# Run simulation with energy tracking
y0 = amazon_net.get_initial_state()
print(f"Initial state shape: {y0.shape}")
print(f"  State variables: {amazon_net.n_elements}")
print(f"  Energy variables: {amazon_net.n_elements}")

# Simulation parameters
duration = 500.0
dt = 0.5
sigma = 0.06  # Moderate noise
alpha = 1.8   # Slightly heavy-tailed (between Gaussian and Lévy)

print(f"\nSimulation parameters:")
print(f"  Duration: {duration}")
print(f"  dt: {dt}")
print(f"  Noise: σ={sigma}, α={alpha}")

# Run
result = energy_constrained_euler_maruyama(
    f_extended=amazon_net.f_extended,
    y0=y0,
    t_span=(0, duration),
    dt=dt,
    sigma=sigma * np.ones(amazon_net.n_elements),
    alpha=alpha * np.ones(amazon_net.n_elements)
)

print(f"\nSimulation complete!")
print(f"  Time points: {len(result.t)}")
print(f"  Final tipped cells: {np.sum(result.x[-1] > 0)} / {amazon_net.n_elements}")

In [ ]:
# Visualize results
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# 1. State evolution (sample cells)
ax1 = axes[0]
sample_cells = [0, 10, 20, 30, 40]
for i in sample_cells:
    ax1.plot(result.t, result.x[:, i], alpha=0.7, linewidth=0.8, label=f'Cell {i}')
ax1.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax1.set_ylabel('State')
ax1.set_title('Sample Cell States Over Time')
ax1.legend(loc='upper right', ncol=5)
ax1.grid(True, alpha=0.3)

# 2. Fraction tipped over time
ax2 = axes[1]
frac_tipped = np.mean(result.x > 0, axis=1)
ax2.plot(result.t, frac_tipped * 100, 'r-', linewidth=2)
ax2.set_ylabel('% Tipped')
ax2.set_title('Cascade Propagation')
ax2.grid(True, alpha=0.3)

# 3. Total system energy
ax3 = axes[2]
E_total = np.sum(result.E, axis=1)
ax3.plot(result.t, E_total, 'g-', linewidth=2)
ax3.set_xlabel('Time')
ax3.set_ylabel('Total Energy')
ax3.set_title('System Energy Evolution')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Spatial cascade visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Snapshots at different times
times = [0, 100, 200, 300, 400, 499]
time_indices = [int(t / dt) for t in times]

for ax, t_idx, t_val in zip(axes.flat, time_indices, times):
    states = result.x[t_idx, :]
    
    # Color by state: green = stable, red = tipped
    colors = ['green' if s < 0 else 'red' for s in states]
    
    ax.scatter(sub_lon, sub_lat, c=colors, s=50, edgecolor='black', linewidth=0.5)
    ax.set_title(f't = {t_val}')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    n_tipped = np.sum(states > 0)
    ax.text(0.02, 0.98, f'{n_tipped}/50 tipped', transform=ax.transAxes,
            verticalalignment='top', fontsize=10,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.suptitle('Spatial Cascade Propagation', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Thermodynamic Analysis

In [ ]:
# Compute thermodynamic metrics
analyzer = EnergyAnalyzer(amazon_net, result)
budget = analyzer.compute_energy_budget()

print("Thermodynamic Analysis")
print("=" * 50)

# Total entropy produced
total_entropy = analyzer.compute_total_entropy_produced()
print(f"Total entropy produced: {total_entropy:.2f}")

# Tipping events
events = analyzer.identify_tipping_events()
tip_events = [e for e in events if e.direction == 'tip']
recover_events = [e for e in events if e.direction == 'recover']

print(f"\nTipping events: {len(tip_events)}")
print(f"Recovery events: {len(recover_events)}")

# Energy costs
if events:
    costs = analyzer.compute_cascade_energy_costs(events)
    print(f"\nAverage entropy per event: {costs['average_entropy_per_event']:.4f}")
    if costs['n_tip_events'] > 0:
        print(f"Tip event entropy (mean): {costs['tip_entropy_avg']:.4f}")
    if costs['n_recover_events'] > 0:
        print(f"Recovery event entropy (mean): {costs['recover_entropy_avg']:.4f}")
    
    if costs['tip_entropy_avg'] > 0 and costs['recover_entropy_avg'] > 0:
        ratio = costs['tip_entropy_avg'] / costs['recover_entropy_avg']
        print(f"\nTip/Recovery ratio: {ratio:.2f}")

In [ ]:
# Entropy production visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Entropy production rate over time
ax1 = axes[0, 0]
ax1.plot(result.t, budget.entropy_production, 'g-', alpha=0.8)
ax1.set_xlabel('Time')
ax1.set_ylabel('Entropy Production Rate')
ax1.set_title('Entropy Production Over Time')
ax1.grid(True, alpha=0.3)

# 2. Cumulative entropy
ax2 = axes[0, 1]
cumulative_entropy = np.cumsum(budget.entropy_production) * dt
ax2.plot(result.t, cumulative_entropy, 'b-', linewidth=2)
ax2.set_xlabel('Time')
ax2.set_ylabel('Cumulative Entropy')
ax2.set_title('Cumulative Entropy Production')
ax2.grid(True, alpha=0.3)

# 3. Energy per cell (final state)
ax3 = axes[1, 0]
final_E = result.E[-1, :]
colors = ['red' if result.x[-1, i] > 0 else 'green' for i in range(50)]
ax3.scatter(sub_lon, sub_lat, c=final_E, cmap='coolwarm', s=50, 
            edgecolor='black', linewidth=0.5)
ax3.set_xlabel('Longitude')
ax3.set_ylabel('Latitude')
ax3.set_title('Final Energy Distribution')
plt.colorbar(ax3.collections[0], ax=ax3, label='Energy')

# 4. Time in tipped state per cell
ax4 = axes[1, 1]
time_tipped = np.mean(result.x > 0, axis=0) * 100
sc = ax4.scatter(sub_lon, sub_lat, c=time_tipped, cmap='RdYlGn_r', s=50,
                 edgecolor='black', linewidth=0.5)
ax4.set_xlabel('Longitude')
ax4.set_ylabel('Latitude')
ax4.set_title('% Time in Tipped State')
plt.colorbar(sc, ax=ax4, label='% Tipped')

plt.tight_layout()
plt.show()

## 6. Identify Thermodynamically Vulnerable Regions

In [ ]:
# Compute vulnerability metrics for each cell
vulnerability_metrics = []

for i in range(amazon_net.n_elements):
    element = amazon_net.get_element(f'cell_{i}')
    
    # Time spent tipped
    time_tipped_pct = np.mean(result.x[:, i] > 0) * 100
    
    # Average state (closer to 0 = more vulnerable)
    avg_state = np.mean(result.x[:, i])
    
    # State variance (high variance = unstable)
    state_var = np.var(result.x[:, i])
    
    # Barrier height (low = vulnerable)
    barrier = element.barrier_height
    
    # Composite vulnerability score
    vulnerability = (time_tipped_pct / 100) * 0.4 + (1 - barrier) * 0.3 + min(state_var, 1) * 0.3
    
    vulnerability_metrics.append({
        'cell': i,
        'lat': element.lat,
        'lon': element.lon,
        'time_tipped': time_tipped_pct,
        'avg_state': avg_state,
        'state_var': state_var,
        'barrier': barrier,
        'vulnerability': vulnerability
    })

# Sort by vulnerability
vulnerability_metrics.sort(key=lambda x: x['vulnerability'], reverse=True)

print("Most Vulnerable Cells")
print("=" * 70)
print(f"{'Cell':>6} {'Lat':>8} {'Lon':>8} {'%Tipped':>8} {'Barrier':>8} {'Vuln':>8}")
print("-" * 70)
for m in vulnerability_metrics[:10]:
    print(f"{m['cell']:>6} {m['lat']:>8.2f} {m['lon']:>8.2f} {m['time_tipped']:>8.1f} {m['barrier']:>8.3f} {m['vulnerability']:>8.3f}")

In [ ]:
# Vulnerability map
fig, ax = plt.subplots(figsize=(12, 8))

vuln_scores = [m['vulnerability'] for m in vulnerability_metrics]
vuln_lons = [m['lon'] for m in vulnerability_metrics]
vuln_lats = [m['lat'] for m in vulnerability_metrics]

# Background: all Amazon cells
ax.scatter(lon, lat, c='lightgray', s=3, alpha=0.3)

# Subnetwork colored by vulnerability
sc = ax.scatter(vuln_lons, vuln_lats, c=vuln_scores, cmap='RdYlGn_r', 
                s=80, edgecolor='black', linewidth=0.5, vmin=0, vmax=1)

# Mark top 5 most vulnerable
for i, m in enumerate(vulnerability_metrics[:5]):
    ax.annotate(f"{i+1}", (m['lon'], m['lat']), fontsize=12, fontweight='bold',
                ha='center', va='center', color='white')

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Thermodynamic Vulnerability Map\n(Red = Most Vulnerable)')
plt.colorbar(sc, label='Vulnerability Score')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Red cells: Low barrier height + frequently tipped = cascade initiation points")
print("- Green cells: High barrier + rarely tipped = resilient regions")
print("- These vulnerability patterns can inform conservation priorities")

## 7. Summary

### Key Findings

1. **Moisture recycling creates spatial coupling** between Amazon grid cells, enabling cascade propagation

2. **Energy tracking reveals thermodynamic costs** of tipping and recovery in spatially-explicit models

3. **Vulnerability mapping** identifies regions most susceptible to cascade initiation

### Data Citation

Amazon moisture recycling data provided by:

> Wunderling, N., Staal, A., Sakschewski, B., Hirota, M., Tuinenburg, O. A., Donges, J. F., Barbosa, H. M. J., & Winkelmann, R. (2022). Recurrent droughts increase risk of cascading tipping events by outpacing adaptive capacities in the Amazon rainforest. *PNAS*, 119(32), e2120777119. https://doi.org/10.1073/pnas.2120777119

Data access: https://figshare.com/articles/software/Amazon_Adaptation_Model/20089331

### Next Steps

1. **Full network analysis**: Scale to all 567 cells using distributed computing
2. **Drought scenarios**: Test cascade response under 2005, 2010, 2015 drought conditions
3. **Lévy vs Gaussian**: Compare noise-type effects in spatial model
4. **Deforestation scenarios**: Model impact of land-use change on cascade vulnerability

In [ ]:
print("Amazon Spatial Energy-Tracking Analysis Complete!")
print("\nData citation: Wunderling et al. (2022) PNAS")
print("Data provided by: Dr. Arie Staal, Utrecht University")